In [2]:
import pandas as pd
import numpy as np
import nltk

### Load in the Data

In [3]:
with open('data/reviews.txt') as f:
    reviews = f.read()
    
with open('data/labels.txt') as f:
    labels = f.read()

In [4]:
print(len(reviews))
print(len(labels))

33678267
225000


In [5]:
print(reviews[:1000])
print()
print(labels[:10])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

### Data Preprocessing

In [6]:
nltk.download('punkt')       # For tokenization
nltk.download('punkt_tab')   # For tokenization
nltk.download('stopwords')   # For stopword removal
nltk.download('wordnet')     # For lemmatization
nltk.download('averaged_perceptron_tagger')  # For part-of-speech tagging
nltk.download('omw-1.4')     # Optional: For extended WordNet data

[nltk_data] Downloading package punkt to C:\Users\Arion
[nltk_data]     Das\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Arion
[nltk_data]     Das\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Arion
[nltk_data]     Das\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Arion
[nltk_data]     Das\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Arion Das\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Arion
[nltk_data]     Das\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is

True

In [7]:

from string import punctuation

# get rid of punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])

# split by new lines and spaces
reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)

# create a list of words
words = all_text.split()

In [8]:
#reviews[:100]

### Encoding the words

In [9]:
from collections import Counter

counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

reviews_ints = []
for review in reviews.split('\n'):
    reviews_ints.append([vocab_to_int[word] for word in review.split() if word in word in vocab_to_int])

In [10]:
print('Unique words: ', len((vocab_to_int)))
print()

print('Tokenized review: \n', reviews_ints[:1])

Unique words:  74072

Tokenized review: 
 [[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]]


In [11]:
labels_split = labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])

In [12]:
print("Length of reviews_ints:", len(reviews_ints))
print("Length of encoded_labels:", len(encoded_labels))

Length of reviews_ints: 25001
Length of encoded_labels: 25001


### Removing Outliers
Reviews of very long / very short length <br>
Padding the remaining data so we have reviews of the same length

In [13]:
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


In [14]:
print('Number of reviews before removing outliers: ', len(reviews_ints))

## remove any reviews/labels with zero length from the reviews_ints list.
reviews_ints = [review for review in reviews_ints if len(review) > 0]
encoded_labels = np.array([encoded_labels[ii] for ii, review in enumerate(reviews_ints) if len(review) > 0])

# get indices of any reviews with length 0
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]

# remove 0-length reviews and their labels
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(reviews_ints))

Number of reviews before removing outliers:  25001
Number of reviews after removing outliers:  25000


### Padding the text

In [15]:
def pad_features(reviews_ints, seq_length):

    features = np.zeros((len(reviews_ints), seq_length), dtype=int)
    
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
        
    return features

In [16]:
seq_length = 200
features = pad_features(reviews_ints, seq_length=seq_length)

assert len(features)==len(reviews_ints)
assert len(features[0]) == seq_length

print(features[:20,:10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34     3   162   748  2731     9   325]
 [    9    11 10171  5305  1946   689   444    22   280   673]
 [    0     0     0     0     0     0     0     0     0

### Train - Test - Validation split

In [17]:
split_frac = 0.8

split_idx = int(len(features)*split_frac)

train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape),
        "\nValidation set: \t{}".format(val_x.shape),
        "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


### Dataloaders & Batching

In [18]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

## creating tensors
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

batch_size = 50

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

### Model

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [20]:
import torch.nn as nn

In [27]:
class SentimentRNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0):
        super(SentimentRNN, self).__init__()
        
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                            dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(0.3)
        
        self.ln = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        #last timestep
        lstm_out = lstm_out[:, -1, :]
        
        out = self.dropout(lstm_out)
        out = self.ln(out)
        sig_out = self.sig(out)
        
        return sig_out, hidden
    
    ## Xavier Initialization
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).to(device))
        
        torch.nn.init.xavier_uniform_(hidden[0])
        torch.nn.init.xavier_uniform_(hidden[1])
        
        return hidden

In [28]:
vocab_size = len(vocab_to_int)+1
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

model = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(model)

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (ln): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


### Training

In [29]:
lr = 0.001

## Binary Cross Entropy Loss for single Sigmoid output
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [30]:
epochs = 4

counter = 0
print_every = 100
clip = 5

model.to(device)

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (ln): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [31]:
for e in range(epochs):
    h = model.init_hidden(batch_size)
    
    for inputs, labels in train_loader:
        counter += 1
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        h = tuple([each.data for each in h])
        
        model.zero_grad()
        
        output, h = model(inputs, h)
        
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        ## loss calculation
        if counter % print_every == 0:
            val_h = model.init_hidden(batch_size)
            val_losses = []
            ## setting the model to eval mode
            model.eval()
            
            for inputs, labels in valid_loader:
                val_h = tuple([each.data for each in val_h])
                
                inputs, labels = inputs.to(device), labels.to(device)
                
                output, val_h = model(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())
                val_losses.append(val_loss.item())
            
            ## setting the model back to train mode
            model.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.634448... Val Loss: 0.651085
Epoch: 1/4... Step: 200... Loss: 0.662099... Val Loss: 0.591812
Epoch: 1/4... Step: 300... Loss: 0.511773... Val Loss: 0.541623
Epoch: 1/4... Step: 400... Loss: 0.594715... Val Loss: 0.551787
Epoch: 2/4... Step: 500... Loss: 0.412505... Val Loss: 0.506731
Epoch: 2/4... Step: 600... Loss: 0.323083... Val Loss: 0.481076
Epoch: 2/4... Step: 700... Loss: 0.419023... Val Loss: 0.455950
Epoch: 2/4... Step: 800... Loss: 0.313172... Val Loss: 0.427427
Epoch: 3/4... Step: 900... Loss: 0.397744... Val Loss: 0.494623
Epoch: 3/4... Step: 1000... Loss: 0.288552... Val Loss: 0.474147
Epoch: 3/4... Step: 1100... Loss: 0.289237... Val Loss: 0.446229
Epoch: 3/4... Step: 1200... Loss: 0.344635... Val Loss: 0.486213
Epoch: 4/4... Step: 1300... Loss: 0.140542... Val Loss: 0.520285
Epoch: 4/4... Step: 1400... Loss: 0.065261... Val Loss: 0.546125
Epoch: 4/4... Step: 1500... Loss: 0.183027... Val Loss: 0.560115
Epoch: 4/4... Step: 1600... Loss: 

### Evaluation

In [32]:
test_losses = []
num_correct = 0

h = model.init_hidden(batch_size)
model.eval()

for inputs, labels in test_loader:
    
    h = tuple([each.data for each in h])
    
    inputs, labels = inputs.to(device), labels.to(device)
    
    output, h = model(inputs, h)
    
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    pred = torch.round(output.squeeze())
    
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)
    
print("Test loss: {:.3f}".format(np.mean(test_losses)))

test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.645
Test accuracy: 0.783


### Inference

In [33]:
test_review_neg = "The worst player of 2024!! \
    Virat Kohli has let his country and himself down; \
    his batting was terrible. He should probably play for RCB only."

In [34]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int.get(word, 0) for word in test_words])

    return test_ints

# test code and generate tokenized review
test_ints = tokenize_review(test_review_neg)
print(test_ints)

[[1, 247, 1757, 4, 0, 0, 0, 47, 278, 27, 657, 2, 309, 180, 27, 24888, 14, 388, 25, 142, 238, 292, 16, 0, 64]]


In [35]:
seq_length=200
features = pad_features(test_ints, seq_length)

print(features)

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0 

In [36]:
feature_tensor = torch.from_numpy(features)
print(feature_tensor.size())

torch.Size([1, 200])


In [37]:
def predict(net, test_review, sequence_length=200):
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    feature_tensor = feature_tensor.to(device)
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")

In [38]:
predict(model, test_review_neg, seq_length)

Prediction value, pre-rounding: 0.003215
Negative review detected.


In [39]:
test_review_pos = "The best player of 2024!! \
    Virat Kohli has made his country and himself proud; \
    his batting was incredible. He should probably play more."

In [40]:
predict(model, test_review_pos, seq_length)

Prediction value, pre-rounding: 0.995640
Positive review detected!
